In [1]:
import sys
sys.path.append('../../')
import numpy as np
import pandas as pd

import typing
from typing import Any, Tuple

import tensorflow as tf
import tensorflow_text as tf_text

import utils.preprocess as pp

2022-10-21 09:09:32.518615: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class ShapeChecker():
    def __init__(self):
    # Keep a cache of every axis-name seen
        self.shapes = {}

    def __call__(self, tensor, names, broadcast=False):
        # __call__ : 클래스의 객체를 호출할 수 있게 만드는 함수 
        if not tf.executing_eagerly():
            return

        if isinstance(names, str):
            names = (names, )

        shape = tf.shape(tensor)
        rank = tf.rank(tensor)

        if rank != len(names):
            raise ValueError(f'Rank mismatch:\n'
                           f'found {rank}: {shape.numpy()}\n'
                           f'expected {len(names)}: {names}\n')

        for i, name in enumerate(names):
            if isinstance(name, int):
                old_dim = name
            else:
                old_dim = self.shapes.get(name, None)
            new_dim = shape[i]

            if (broadcast and new_dim == 1):
                continue

            if old_dim is None:
            # If the axis name is new, add its length to the cache.
                self.shapes[name] = new_dim
                continue

            if new_dim != old_dim:
                raise ValueError(f"Shape mismatch for dimension: '{name}'\n"
                             f"found: {new_dim}\n"
                             f"expected: {old_dim}\n")

In [3]:
df = pd.read_csv('../../data/sample/colloquial_correct_train_data.csv')

In [4]:
df.head(1)

,Unnamed: 0,src,tgt,src_morphs_ko,src_morphs_length,src_replace_morphs_text,src_tokenizing_text,src_morphs_ko.1,src_morphs_length.1,tgt_morphs_ko,tgt_morphs_length,tgt_replace_morphs_text,tgt_tokenizing_text
0,0,"네, 언제든지 편하실 때 체크아우타시면 도와드릴게요.","네, 언제든지 편하실 때 체크아웃하시면 도와드릴게요.","['네', ',', '▁', '언제', '든지', '▁', '편하', '실', '▁...",20.0,"네 , ▁ 언제 든지 ▁ 편하 실 ▁ 때 ▁ 체크 아우 타 시 면 ▁ 도와 드릴게요 .","▁네▁,▁▁언제▁든지▁▁편하▁실▁▁때▁▁체크▁아우▁타▁시▁면▁▁도와▁드릴게요▁.","['네', ',', '▁', '언제', '든지', '▁', '편하', '실', '▁...",20,"['네', ',', '▁', '언제', '든지', '▁', '편하', '실', '▁...",19,"네 , ▁ 언제 든지 ▁ 편하 실 ▁ 때 ▁ 체크아웃 하 시 면 ▁ 도와 드릴게요 .","▁네▁,▁▁언제▁든지▁▁편하▁실▁▁때▁▁체크아웃▁하▁시▁면▁▁도와▁드릴게요▁."


In [5]:
df.columns

Index(['Unnamed: 0', 'src', 'tgt', 'src_morphs_ko', 'src_morphs_length',
       'src_replace_morphs_text', 'src_tokenizing_text', 'src_morphs_ko.1',
       'src_morphs_length.1', 'tgt_morphs_ko', 'tgt_morphs_length',
       'tgt_replace_morphs_text', 'tgt_tokenizing_text'],
      dtype='object')

In [6]:
df = df.drop(['Unnamed: 0','src_morphs_ko.1', 'src_morphs_length.1'], axis=1)
df

,src,tgt,src_morphs_ko,src_morphs_length,src_replace_morphs_text,src_tokenizing_text,tgt_morphs_ko,tgt_morphs_length,tgt_replace_morphs_text,tgt_tokenizing_text
0,"네, 언제든지 편하실 때 체크아우타시면 도와드릴게요.","네, 언제든지 편하실 때 체크아웃하시면 도와드릴게요.","['네', ',', '▁', '언제', '든지', '▁', '편하', '실', '▁...",20.0,"네 , ▁ 언제 든지 ▁ 편하 실 ▁ 때 ▁ 체크 아우 타 시 면 ▁ 도와 드릴게요 .","▁네▁,▁▁언제▁든지▁▁편하▁실▁▁때▁▁체크▁아우▁타▁시▁면▁▁도와▁드릴게요▁.","['네', ',', '▁', '언제', '든지', '▁', '편하', '실', '▁...",19,"네 , ▁ 언제 든지 ▁ 편하 실 ▁ 때 ▁ 체크아웃 하 시 면 ▁ 도와 드릴게요 .","▁네▁,▁▁언제▁든지▁▁편하▁실▁▁때▁▁체크아웃▁하▁시▁면▁▁도와▁드릴게요▁."
1,성장하는 재판매 사업짜 그루베 고갱니믈 초대하고 십씀니다.,성장하는 재판매 사업자 그룹에 고객님을 초대하고 싶습니다.,"['성장', '하', '는', '▁', '재판매', '▁', '사업', '짜', '...",23.0,성장 하 는 ▁ 재판매 ▁ 사업 짜 ▁ 그루베 ▁ 고 갱 니 믈 ▁ 초대 하 고 ▁...,▁성장▁하▁는▁▁재판매▁▁사업▁짜▁▁그루베▁▁고▁갱▁니▁믈▁▁초대▁하▁고▁▁십▁씀니다▁.,"['성장', '하', '는', '▁', '재판매', '▁', '사업자', '▁', ...",22,성장 하 는 ▁ 재판매 ▁ 사업자 ▁ 그룹 에 ▁ 고객 님 을 ▁ 초대 하 고 ▁ ...,▁성장▁하▁는▁▁재판매▁▁사업자▁▁그룹▁에▁▁고객▁님▁을▁▁초대▁하▁고▁▁싶▁습니다▁.
2,"안녕하세요, 예야카려고 전화를 드려써요.","안녕하세요, 예약하려고 전화를 드렸어요.","['안녕', '하', '세요', ',', '▁', '예', '야', '카', '려고...",16.0,"안녕 하 세요 , ▁ 예 야 카 려고 ▁ 전화 를 ▁ 드려 써요 .","▁안녕▁하▁세요▁,▁▁예▁야▁카▁려고▁▁전화▁를▁▁드려▁써요▁.","['안녕', '하', '세요', ',', '▁', '예약', '하', '려고', '...",15,"안녕 하 세요 , ▁ 예약 하 려고 ▁ 전화 를 ▁ 드렸 어요 .","▁안녕▁하▁세요▁,▁▁예약▁하▁려고▁▁전화▁를▁▁드렸▁어요▁."
3,손니미 완는데 방이 업쓰며 너떠캐요?,손님이 왔는데 방이 없으면 어떡해요?,"['손니', '미', '▁', '완', '는데', '▁', '방', '이', '▁'...",18.0,손니 미 ▁ 완 는데 ▁ 방 이 ▁ 업 쓰 며 ▁ 너 떠 캐 요 ?,▁손니▁미▁▁완▁는데▁▁방▁이▁▁업▁쓰▁며▁▁너▁떠▁캐▁요▁?,"['손', '님', '이', '▁', '왔', '는데', '▁', '방', '이',...",16,손 님 이 ▁ 왔 는데 ▁ 방 이 ▁ 없 으면 ▁ 어떡 해요 ?,▁손▁님▁이▁▁왔▁는데▁▁방▁이▁▁없▁으면▁▁어떡▁해요▁?
4,"아니요, 시간 낭비예요, 다시느 녀기에 아 놀 꺼예요.","아니요, 시간 낭비예요, 다시는 여기에 안 올 거예요.","['아니요', ',', '▁', '시간', '▁', '낭비', '예요', ',', ...",22.0,"아니요 , ▁ 시간 ▁ 낭비 예요 , ▁ 다시 느 ▁ 녀 기에 ▁ 아 ▁ 놀 ▁ 꺼...","▁아니요▁,▁▁시간▁▁낭비▁예요▁,▁▁다시▁느▁▁녀▁기에▁▁아▁▁놀▁▁꺼▁예요▁.","['아니요', ',', '▁', '시간', '▁', '낭비', '예요', ',', ...",22,"아니요 , ▁ 시간 ▁ 낭비 예요 , ▁ 다시 는 ▁ 여기 에 ▁ 안 ▁ 올 ▁ 거...","▁아니요▁,▁▁시간▁▁낭비▁예요▁,▁▁다시▁는▁▁여기▁에▁▁안▁▁올▁▁거▁예요▁."
...,...,...,...,...,...,...,...,...,...,...
49995,NaN,NaN,NaN,NaN,NaN,NaN,"['어떤', '▁', '다른', '▁', '서류', '가', '▁', '필요', '...",21,어떤 ▁ 다른 ▁ 서류 가 ▁ 필요 하 신지 ▁ 다시 ▁ 말씀 해 ▁ 주 시 겠 습...,▁어떤▁▁다른▁▁서류▁가▁▁필요▁하▁신지▁▁다시▁▁말씀▁해▁▁주▁시▁겠▁습니까▁?
49996,NaN,NaN,NaN,NaN,NaN,NaN,"['비행기', '에서', '▁', '담배', '를', '▁', '피울', '▁', ...",15,비행기 에서 ▁ 담배 를 ▁ 피울 ▁ 생각 은 ▁ 없 었 어요 .,▁비행기▁에서▁▁담배▁를▁▁피울▁▁생각▁은▁▁없▁었▁어요▁.
49997,NaN,NaN,NaN,NaN,NaN,NaN,"['전', '▁', '오래', '▁', '쓸', '▁', '수', '▁', '있',...",16,전 ▁ 오래 ▁ 쓸 ▁ 수 ▁ 있 는 ▁ 게 ▁ 필요 해요 .,▁전▁▁오래▁▁쓸▁▁수▁▁있▁는▁▁게▁▁필요▁해요▁.
49998,NaN,NaN,NaN,NaN,NaN,NaN,"['털', '을', '▁', '강화', '하', '고', '▁', '앉', '았',...",24,털 을 ▁ 강화 하 고 ▁ 앉 았 을 ▁ 때 ▁ 반짝이 고 ▁ 푹신 한 ▁ 느낌 을...,▁털▁을▁▁강화▁하▁고▁▁앉▁았▁을▁▁때▁▁반짝이▁고▁▁신▁한▁▁느낌▁을▁▁줍니다▁.


In [7]:
max(df['src_morphs_length'])

24.0